In [2]:
# This notebook takes in the philadelphia tax assements and census tract datasets and figures out
# which addresses are most likely to qualify for tax abatement.
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmplot

opa_properties_public = None;
historical_assessments = None;
census_data = None;


try:
    opa_properties_public = pd.read_csv("opa_properties_public.csv")
except FileNotFoundError:
    urllib.request.urlretrieve ("https://phl.carto.com/api/v2/sql?q=SELECT+*,+ST_Y(the_geom)+AS+lat,+ST_X(the_geom)+AS+lng+FROM+opa_properties_public&filename=opa_properties_public&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator", "opa_properties_public.csv")
    opa_properties_public = pd.read_csv("opa_properties_public.csv")
    
try:
    census_data = pd.read_csv("census_data.csv")
except FileNotFoundError:
    urllib.request.urlretrieve ("http://data.phl.opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.csv", "census_data.csv")
    census_data = pd.read_csv("census_data.csv")
    
try:
    historical_assessments = pd.read_csv("historical_assessments.csv")
except FileNotFoundError:
    urllib.request.urlretrieve("https://phl.carto.com/api/v2/sql?q=SELECT+*+FROM+assessments&filename=assessments&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator", "historical_assessments.csv")
    historical_assessments = pd.read_csv("historical_assessments.csv")


C:\Users\steph\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,3,10,28,29,30,31,32,33,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#This function will return the set of houses which might qualify for homestead exemption
def getOwnerOccupied(ownerOccupied):
    print(len(ownerOccupied.index))
    
    
    # We filter out houses where the owner is a company or the city. Trusts and companies are never eligible
    # for the homestead exemption.
    ownerOccupied = ownerOccupied.loc[~ownerOccupied['owner_1'].str.contains('phila|Phila|LLC|llc|CORP|LP|COMP|INC|NEIGHBORHOOD|CAPITAL|INVESTMENTS|LIMITED|LIABILITY', na=False)]
    print(len(ownerOccupied.index))
    
    #Filter out properties that are vacant lots
    ownerOccupied = ownerOccupied.loc[ownerOccupied['category_code_description'] != 'Vacant Land']
    print(len(ownerOccupied.index))
    
    #Filter out properties that are not residential zoned
    ownerOccupied = ownerOccupied.loc[ownerOccupied['zoning'].str.startswith(('R', 'IRMX', 'CMX'), na=False)]
    print(len(ownerOccupied.index))
    
    ownerOccupied = ownerOccupied.loc[ownerOccupied['homestead_exemption'] == 0]
    print(len(ownerOccupied.index))
    return ownerOccupied
    
getOwnerOccupied(opa_properties_public.copy())

581328
518358
486421
480385
255232


,number_of_rooms,assessment_date,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,...,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,objectid,lat,lng
0,6.0,NaN,"153'7 1/2""N ALLEGHENY",1.77592e+06,O30,ROW 2 STY MASONRY,1.0,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342205,RM1,572233332,39.998934,-75.127498
1,6.0,NaN,"16'4 1/2""S WILLARD",1.74793e+06,O30,ROW 2 STY MASONRY,1.0,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342205,RM1,572233333,39.999272,-75.127344
2,6.0,NaN,SWC WILLARD,3.50011e+06,O30,ROW 2 STY MASONRY,1.0,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342205,RM1,572233334,39.999317,-75.127335
5,6.0,NaN,"80'6"" S WESTMORELAND",2.6401e+06,O30,ROW 2 STY MASONRY,1.0,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342227,RM1,572233337,39.999601,-75.127268
6,6.0,NaN,"16'4 1/2""N WILLARD",3.52303e+06,O30,ROW 2 STY MASONRY,1.0,Single Family,520.0,NaN,...,NaN,NaN,I,1935,Y,191342226,RM1,572233338,39.999453,-75.126881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581311,6.0,NaN,31.854' S CAMBRIDGE ST,3289609,O50,ROW 3 STY MASONRY,1.0,Single Family,250.0,Y,...,NaN,NaN,I,2011,NaN,191232270,RSA5,572811370,39.968957,-75.142610
581313,6.0,NaN,"76'11"" W LAWRENCE ST",3501941,O30,ROW 2 STY MASONRY,1.0,Single Family,621.0,NaN,...,NaN,NaN,I,1945,Y,191404550,RM1,572811372,40.007625,-75.136242
581314,6.0,NaN,98'E FRONTENAC,3081425,J30,S/D W/D GAR 2 STY MASONRY,1.0,Single Family,10.0,N,...,NaN,NaN,I,1950,Y,191114217,RSA3,572811583,40.051348,-75.070933
581315,0.0,NaN,"28'1 6/8"" N OLD UMBRIA",2780982,551,RES CONDO 3 STY MAS+OTH,1.0,Single Family,541.0,Y,...,0000002,NaN,I,2005,Y,191271913,CMX2,572811584,40.030184,-75.230018


In [38]:
#This method tries to find homes that are eligible and would benefit from LOOP
def getLOOP(data): 
#     parcel_nums = pd.DataFrame({'parcel_number': data['parcel_number']})
#     #print(parcel_nums)
    
    this_year = data.loc[data['year'] == 2020]
    this_year = this_year.drop(columns=['year', 'taxable_land', 'taxable_building',
                                        'exempt_land', 'exempt_building'])
    this_year = this_year.rename(columns={'market_value': 'this_year'})
    
    if(len(this_year['parcel_number']) == len(set(this_year['parcel_number']))):
        print("DUPS")
    else:
        print("NO DUPS")
    
    last_year = data.loc[data['year'] == 2019]
    last_year = last_year.drop(columns=['year', 'taxable_land', 'taxable_building',
                                        'exempt_land', 'exempt_building'])
    last_year = last_year.rename(columns={"market_value": "last_year"})
    print(this_year)
    
    #diff = this_year.set_index('parcel_number').join(last_year.set_index('parcel_number'), how='inner')
    #print(diff)
    
    diff = pd.merge(this_year, last_year, on='parcel_number', how='inner')
    
    #Loop limits tax increase to 50% after the homestead exemption is taken. 
    loop_eligible = diff.loc[(diff['this_year'] - 45000) > (1.5 * diff['last_year'])]
    
    #Finally, we need to fix the parcel_numbers so they are integers again
    return loop_eligible

#NOTE: Not all price increases are due to loop reasons. Home improvment and building are one as well
def graphLOOPGmaps(loop_eligible, opa_properties_public): 
    #We get rid of elements where last year was 0 (thus empty lot) was improved
    loop_eligible = loop_eligible.loc[loop_eligible['last_year'] != 0]
    
    #First we combine, and then get the latitiudes and longitudes
    combined = pd.merge(loop_eligible, opa_properties_public, on='parcel_number', how='inner')
    
    #The api key lives outside the repo for security
    #TODO: DEFINE GOOGLE API KEY
    mykey = ""
    gmap = gmplot.GoogleMapPlotter(39.952583, -75.165222, 10)
    gmap.apikey=mykey
    gmap.heatmap(combined['lat'], combined['lng'])
    gmap.draw("phila_loop_heatmap.html")
    
loop_eligible = getLOOP(historical_assessments)

def graphLOOPOpenStreet(loop_eligible, opa_properties_public):
    #We get rid of elements where last year was 0 (thus empty lot) was improved
    loop_eligible = loop_eligible.loc[loop_eligible['last_year'] != 0]
    bounds = (opa_properties_public['lng'].min(), opa_properties_public['lng'].max(),      
         opa_properties_public['lat'].min(), opa_properties_public['lat'].max())
    print(bounds)

    
  
graphLOOP(loop_eligible, opa_properties_public)    
print(loop_eligible)
graphLOOPOpenStreet(loop_eligible, opa_properties_public)

NO DUPS
         parcel_number  this_year
73275       12334300.0   139200.0
73276       12294500.0   238900.0
73282       12334400.0   210000.0
73283       12295000.0   240200.0
73289       12334500.0   207400.0
...                ...        ...
2467372    888560708.0   162600.0
2467373    888560367.0   146200.0
2467379    888560709.0   146200.0
2467380    888560368.0   166700.0
2467386    888560710.0   162600.0

[581325 rows x 2 columns]
        parcel_number  this_year  last_year
180        11281000.0   311000.0   167500.0
348        12139680.0   448300.0    26300.0
730        11130500.0   193400.0    69700.0
871        11137300.0   433000.0   151100.0
990        12233100.0   497800.0   218800.0
...               ...        ...        ...
566096    888582053.0   141900.0    23100.0
568016    888680026.0  3629000.0        0.0
571837    888501135.0   189000.0        0.0
576443    888091421.0   774000.0   279100.0
576531    888095271.0  2968800.0  1785700.0

[2658 rows x 3 columns]
(-75